In [1]:
import os, shutil, glob, time
import pandas as pd
os.environ["HADOOP_HOME"] = "D:/hadoop"
os.environ["PATH"] += os.pathsep + "D:/hadoop/bin"
os.makedirs("D:/hadoop/checkpoint", exist_ok=True)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, from_unixtime
from pyspark.sql.types import StructType, IntegerType, DoubleType, TimestampType
import threading

In [3]:
# Cấu hình thư mục
output_dir = "D:/sensor-data/output"
checkpoint_dir = "D:/hadoop/checkpoint"
backup_dir = "D:/sensor-data/backup"

# Tạo thư mục nếu chưa có
os.makedirs(output_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(backup_dir, exist_ok=True)


def merge_csv_files_clean_headers(input_dir, output_file):
    csv_files = glob.glob(os.path.join(input_dir, "part-*"))
    if not csv_files:
        print("⚠️ Không có file để gộp.")
        return

    dfs = []
    for file in csv_files:
        try:
            df = pd.read_csv(file, header=None)  # Không có header
            # Lọc bỏ các dòng tiêu đề trùng (ví dụ: dòng chứa 'sensor_id')
            df = df[df[0] != 'sensor_id']
            dfs.append(df)
        except Exception as e:
            print(f"❌ Lỗi đọc file {file}: {e}")

    if dfs:
        merged_df = pd.concat(dfs, ignore_index=True)
        # Gán lại tên cột chuẩn
        merged_df.columns = ['sensor_id', 'temperature', 'humidity', 'timestamp']
        merged_df.to_csv(output_file, index=False)
        print(f"✅ Đã gộp và làm sạch thành {output_file}")
    else:
        print("⚠️ Không có nội dung hợp lệ để gộp.")


# Hàm xoá thư mục cũ
def cleanup_dirs():
    shutil.rmtree(output_dir, ignore_errors=True)
    shutil.rmtree(checkpoint_dir, ignore_errors=True)
    print("🧹 Đã xoá output và checkpoint.")


In [4]:
# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("KafkaSensorConsumerAuto") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2") \
    .config("spark.hadoop.home.dir", "D:/hadoop") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# Schema dữ liệu JSON
schema = StructType() \
    .add('sensor_id', IntegerType()) \
    .add('temperature', DoubleType()) \
    .add('humidity', DoubleType()) \
    .add('timestamp', DoubleType())  

# Đọc dữ liệu từ Kafka
df_raw = spark.readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers', 'localhost:9092') \
    .option('subscribe', 'sensor-data') \
    .option('startingOffsets', 'latest') \
    .load()

df_parsed = df_raw.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*") \
    .withColumn("timestamp", from_unixtime(col("timestamp")).cast("string"))  

# Ghi dữ liệu vào file CSV
query = df_parsed.writeStream \
    .option("path", output_dir) \
    .option("checkpointLocation", checkpoint_dir) \
    .option("header", True) \
    .format("csv") \
    .start()

query.awaitTermination(120)  # 120 giây = 2 phút
query.stop()
print("🔴 Đã dừng stream sau 2 phút.")



🔴 Đã dừng stream sau 2 phút.


In [5]:
timestamp_str = time.strftime("%Y%m%d_%H%M%S")
merged_file_path = f"{backup_dir}/merged_{timestamp_str}.csv"
merge_csv_files_clean_headers(output_dir, merged_file_path)


✅ Đã gộp và làm sạch thành D:/sensor-data/backup/merged_20250615_125608.csv


In [6]:
cleanup_dirs()

🧹 Đã xoá output và checkpoint.
